In [7]:
using JaynesCummings, PlotlyJS

In [44]:
N = 6
ω_q = 2π*6.57E9
ω_r = ω_q*0.95
Δ = abs(ω_q-ω_r)
g = 0.5*2π*19E6
ω_d = ω_q+g^2/Δ
ϵ = 0.03ω_d
time = 3π*Δ/(2*g*ϵ)
samples = 50
ket = ["|g,0>"] # ground state
@printf("Δ (MHz) = %f\ng/Δ = %f\nϵ*g/Δ (MHz) = %f\nTime (ns): %f\nPeriods: %f",
         Δ/2π/1E6,g/Δ,ϵ*g/(2π*Δ*1E6),time*1E9,time/((2π*Δ)/(2*g*ϵ)))

Δ (MHz) = 328.500000
g/Δ = 0.028919
ϵ*g/Δ (MHz) = 5.700238
Time (ns): 131.573445
Periods: 1.500000

In [45]:
# Generate the initial (ground) state and the Hamiltonian
ρ = gen_initialstate(N,ket)
H = gen_hamiltonian(N,ω_q,ω_r,g,rwa=false);

In [46]:
# Add the drive and generate time evolution operators
Δt, H_d_array = gen_drive_hamiltonian(ω_d,N,samples)
H_array = [H+ϵ*H_d for H_d in H_d_array]
U_rel_array = map(H_array.*Δt) do HΔt
    expm(-1.0im * HΔt / ħ)
end;

In [47]:
# We now are in possession of the propagators U for a full drive period.
# We can time-evolve the initial state by applying U in succession.
steps = round(Int,time/Δt)
e_prob = calc_qubittimeevo_rel(ρ,U_rel_array,steps);

In [48]:
# Plot qubit state over time
times = (1:length(e_prob))*Δt*10E9
plot([scatter(;x=times,y=1-e_prob,name="Ground State Prob"),
    scatter(;x=times,y=e_prob,name="Excited State Prob")],
    Layout(;xaxis=attr(title="Drive time (ns)")))